In [1]:
!ls
!ls data

data             task4_2005.ipynb
food               sample.txt         train_triplets.txt
food.zip           test_triplets.txt


In [2]:
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from keras.applications import MobileNetV2
from keras.applications import VGG16




Using TensorFlow backend.


In [3]:
print(tf.__version__)
print(keras.__version__)

2.0.0
2.2.4-tf


In [4]:
model = MobileNetV2()

In [5]:
model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
_______________________________________________________________________________

In [ ]:
#model_vgg = VGG16(include_top=True)
#model_vgg.summary()

In [28]:
import pandas as pd

sample = pd.read_table("data/sample.txt",header=None)
train_triplets = pd.read_table("data/train_triplets.txt",delimiter=" ",header=None,dtype=str)
test_triplets = pd.read_table("data/test_triplets.txt",delimiter=" ",header=None,dtype=str)

In [29]:
train_triplets


,0,1,2
0,02461,03450,02678
1,02299,02499,04987
2,04663,01056,03029
3,04532,01186,01297
4,03454,03809,02204
...,...,...,...
59510,00466,02952,02530
59511,02646,03580,02359
59512,03255,04844,04334
59513,02136,04619,00161


In [30]:
test_triplets

,0,1,2
0,09896,09640,09177
1,06592,09283,07104
2,08655,06174,06400
3,09223,08187,08678
4,07317,05392,09470
...,...,...,...
59539,06113,08042,06277
59540,08851,06075,08549
59541,06299,07843,07940
59542,07652,05620,05416


In [36]:
test_triplets[0][1]

'06592'

In [43]:
def image_pred(image_code):
    image_path = 'data/food/'+image_code+'.jpg'
    image = load_img(image_path, target_size=(224, 224))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    yhat = (model.predict(image)).flatten()

    return(yhat)


def triplet_pred(triplet):
    triplet_feats = np.zeros((3,1000))
    for i in range(len(triplet)):
        triplet_feats[i] = image_pred(triplet[i])

    if np.dot(triplet_feats[0],triplet_feats[1]) > np.dot(triplet_feats[0],triplet_feats[2]):
        return(1)
    else:
        return(0)


In [46]:
train_triplets.shape

(59515, 3)

In [49]:
n = train_triplets.shape[0]
y = np.zeros(n)

for i in range(100):
    triplet = train_triplets.loc[i].values
    y[i] = triplet_pred(triplet)
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [51]:
np.sum(y[:100])

59.0

In [6]:

from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.mobilenet import preprocess_input
#from keras.applications.vgg16 import decode_predictions
#from keras.applications.vgg16 import VGG16
# load an image from file
image = load_img('data/food/00000.jpg', target_size=(224, 224))
# convert the image pixels to a numpy array
image = img_to_array(image)
# reshape data for the model
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
# prepare the image for the VGG model
image = preprocess_input(image)

In [8]:
yhat = model.predict(image)

In [37]:
np.shape(yhat.flatten())

(1000,)